## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-22-40-54 +0000,bbc,An Indigenous Australian community is fighting...,https://www.bbc.com/news/articles/ckglj211x9yo
1,2025-07-11-22-34-55 +0000,nyt,"Brian Clarke, Stained-Glass Innovator, Is Dead...",https://www.nytimes.com/2025/07/11/arts/design...
2,2025-07-11-22-34-41 +0000,nypost,"Fannie, Freddie boss Bill Pulte claims Jerome ...",https://nypost.com/2025/07/11/business/fannie-...
3,2025-07-11-22-28-20 +0000,bbc,Trade trumps geopolitics as Australia PM visit...,https://www.bbc.com/news/articles/c7vrjlp5zdvo
4,2025-07-11-22-10-00 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2294/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,15
47,tariff,5
120,report,3
130,canada,3
160,will,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,30
38,2025-07-11-13-46-34 +0000,nypost,"Dow tumbles, S&P 500 slips from record high af...",https://nypost.com/2025/07/11/business/sampp-5...,30
5,2025-07-11-21-50-57 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...,29
52,2025-07-11-01-45-42 +0000,nyt,Trump Tariff Letters: See the Latest Rates and...,https://www.nytimes.com/2025/07/07/business/tr...,28
19,2025-07-11-19-04-37 +0000,nyt,Trump Says NATO Countries Will Buy Weapons to ...,https://www.nytimes.com/2025/07/11/world/europ...,27


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,30,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
29,19,2025-07-11-16-28-54 +0000,nypost,NBC’s ‘Nightly News’ with Tom Llamas beats ABC...,https://nypost.com/2025/07/11/media/nbcs-night...
17,15,2025-07-11-19-33-00 +0000,nypost,Reddit flooded with antisemitic conspiracy the...,https://nypost.com/2025/07/11/business/reddit-...
26,15,2025-07-11-17-14-01 +0000,nypost,Unilever appoints new Ben & Jerry’s CEO as bat...,https://nypost.com/2025/07/11/business/unileve...
2,14,2025-07-11-22-34-41 +0000,nypost,"Fannie, Freddie boss Bill Pulte claims Jerome ...",https://nypost.com/2025/07/11/business/fannie-...
8,14,2025-07-11-21-35-14 +0000,nypost,Skydance reportedly in talks to buy Bari Weiss...,https://nypost.com/2025/07/11/media/skydance-r...
54,14,2025-07-10-23-27-50 +0000,nypost,Jamie Dimon lashes out at ‘Marxist’ Zohran Mam...,https://nypost.com/2025/07/10/business/jamie-d...
56,14,2025-07-10-22-59-48 +0000,nypost,Interest will restart for 8 million student lo...,https://nypost.com/2025/07/10/business/interes...
48,12,2025-07-11-08-14-13 +0000,bbc,'I created it myself': The boy behind the vira...,https://www.bbc.com/news/articles/czxe7ey59r0o
4,11,2025-07-11-22-10-00 +0000,bbc,Arizona wildfires and extreme heat prompt eva...,https://www.bbc.com/news/articles/czjk10wwmkeo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
